In [3]:
import numpy as np
import pandas as pd
import re

In [4]:
df = pd.read_csv("HillaryEmails.csv")

df = df[['Id','ExtractedBodyText']].dropna()

In [5]:
def clean_email_text(text):
    text = text.replace('\n'," ")
    text = re.sub(r"-", " ", text) #把 "-" 的左右两个词分开
    text = re.sub(r"\d+/\d+/\d+", "", text) #日期
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text) #时间
    text = re.sub(r"[\w]+@[\.\w]+", "", text) #邮件地址
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text) #网址
    pure_text = ''
    # 以防还有其他特殊字符（数字）等等，我们直接把他们loop一遍，过滤掉
    for letter in text:
        # 只留下字母和空格
        if letter.isalpha() or letter==' ':
            pure_text += letter
    # 再把那些去除特殊字符后落单的单词，直接排除。
    # 我们就只剩下有意义的单词了。
    text = ' '.join(word for word in pure_text.split() if len(word)>1)
    return text

In [6]:
docs = df['ExtractedBodyText']
docs = docs.apply(lambda s: clean_email_text(s))  

In [7]:
doclist = docs.values

In [8]:
from gensim import corpora, models, similarities
import gensim

In [9]:
stoplist = ['very', 'ourselves', 'am', 'doesn', 'through', 'me', 'against', 'up', 'just', 'her', 'ours', 
            'couldn', 'because', 'is', 'isn', 'it', 'only', 'in', 'such', 'too', 'mustn', 'under', 'their', 
            'if', 'to', 'my', 'himself', 'after', 'why', 'while', 'can', 'each', 'itself', 'his', 'all', 'once', 
            'herself', 'more', 'our', 'they', 'hasn', 'on', 'ma', 'them', 'its', 'where', 'did', 'll', 'you', 
            'didn', 'nor', 'as', 'now', 'before', 'those', 'yours', 'from', 'who', 'was', 'm', 'been', 'will', 
            'into', 'same', 'how', 'some', 'of', 'out', 'with', 's', 'being', 't', 'mightn', 'she', 'again', 'be', 
            'by', 'shan', 'have', 'yourselves', 'needn', 'and', 'are', 'o', 'these', 'further', 'most', 'yourself', 
            'having', 'aren', 'here', 'he', 'were', 'but', 'this', 'myself', 'own', 'we', 'so', 'i', 'does', 'both', 
            'when', 'between', 'd', 'had', 'the', 'y', 'has', 'down', 'off', 'than', 'haven', 'whom', 'wouldn', 
            'should', 've', 'over', 'themselves', 'few', 'then', 'hadn', 'what', 'until', 'won', 'no', 'about', 
            'any', 'that', 'for', 'shouldn', 'don', 'do', 'there', 'doing', 'an', 'or', 'ain', 'hers', 'wasn', 
            'weren', 'above', 'a', 'at', 'your', 'theirs', 'below', 'other', 'not', 're', 'him', 'during', 'which']

In [10]:
texts = [[word for word in doc.lower().split() if word not in stoplist] for doc in doclist]

In [11]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [12]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

In [13]:
lda.print_topic(10, topn=5)

'0.013*"call" + 0.009*"add" + 0.006*"list" + 0.005*"back" + 0.004*"would"'

In [14]:
df = pd.read_csv("HillaryTwitter.csv")

df = df.dropna()

In [15]:
docs = df['test']
docs = docs.apply(lambda s: clean_email_text(s))  

In [16]:
docs.head(1).values

array([ 'To all the little girls watchingnever doubt that you are valuable and powerful deserving of every chance opportunity in the world'], dtype=object)

In [17]:
doclist = docs.values

In [18]:
texts = [[word for word in doc.lower().split() if word not in stoplist] for doc in doclist]

In [19]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [20]:
corpus[0],texts[0]

([(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1)],
 ['little',
  'girls',
  'watchingnever',
  'doubt',
  'valuable',
  'powerful',
  'deserving',
  'every',
  'chance',
  'opportunity',
  'world'])

In [21]:
np.array(lda.get_document_topics(corpus[1]))[:,0]

array([ 0.,  4.])

In [57]:
def get_topic(tweets):
    if tweets >14:
        print('输入范围在[0,14]之间')
    else:
        corpus_num = np.array(lda.get_document_topics(corpus[tweets]))[:, 0]
        print("希拉里推特第%d句话，分别属于第%s个主题" % (tweets+1, corpus_num))

In [59]:
get_topic(2)

希拉里推特第3句话，分别属于第[  0.   7.  15.]个主题
